In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display,HTML
import pandasql as ps 

class Web_Scrapper:

    def __init__(self) -> None:
        pass

# crea lista de urls
    def list_maker_attribute(self,repeticion,lista,tag_name,attribute) -> list:
        """Crea una lista de valores de atributos, extrae elementos de otra lista identidicados por TAG_NAME, util para obtener URL. 

        Args:
            repeticion (_int_): Numero de veces que se itera
            lista (_list_): Lista que contiene los elementos html
            tag_name (_str_): Localizador tag (html)
            attribute (_str_): Atributo del que se desea obtener valor

        Returns:
            list: Lista de los valores del atributo deseado
        """
        lista_final=[]
        for i in range(repeticion):
            try:
                lista_final.append(lista[i].find_element(By.TAG_NAME,tag_name).get_attribute(attribute))
            except Exception:
                lista_final.append(np.nan)
        return lista_final

# crea lista de un elemento html especifico indicado por class name
    def list_maker_class(self,repeticion,lista,class_name) -> list:
        """Crea una lista de valores de atributos, extrae elementos de otra lista identidicados por CLASS_NAME, util para obtener contenido de elementos html. 

        Args:
            repeticion (_int_): Numero de veces que se itera
            lista (_list_): Lista que contiene los elementos html
            class_name (_str_): Valor del atributo class (html)

        Returns:
            list: Lista de los valores del atributo deseado (class)
        """
        lista_final=[]
        for i in range(repeticion):
            try:
                lista_final.append(lista[i].find_element(By.CLASS_NAME,class_name).text)
            except Exception:
                lista_final.append(np.nan)
        return lista_final    
    
# crea lista de un elemento html especifico indicado por class name dentro de un class name
    def list_maker_class2(self,repeticion,lista,class_name,class_name2) -> list:
        """Crea una lista de valores de atributos, extrae elementos de otra lista identidicados por CLASS_NAME dos ocasiones, util para obtener contenido de elementos html. 

        Args:
            repeticion (_int_): Numero de veces que se itera
            lista (_list_): Lista que contiene los elementos html
            class_name (_str_): Valor del atributo class (html)
            class_name2 (_str_): Valor del segundo atributo class (html)

        Returns:
            list: Lista de los valores del atributo deseado (class)
        """
        lista_final=[]
        for i in range(repeticion):
            try:
                lista_final.append(lista[i].find_element(By.CLASS_NAME,class_name).find_element(By.CLASS_NAME,class_name2).text)
            except Exception:
                lista_final.append(np.nan)
        return lista_final

    def delete_extra_info(self,lista,caracter) -> list:
        """Extrae substring de un string con metodo split, conserva la particion de interes.

        Args:
            lista (_list_): Lista con elementos que se quiere reducir
            caracter (_str_): Cadena a partir de la cual se haran particiones

        Returns:
            list: Lista con elementos reducidos
        """
        for i in range(len(lista)):
            lista[i]=lista[i].split(caracter)[0]
        return lista

    def to_binary(self,lista,condicion) -> list:
        """A partir de una condicion, sustituye por 1 si la condicion es cierta, falso en otro caso para los elementos de una lista.

        Args:
            lista (_list_): Lista sin estandarizar
            condicion (_str_): Condicion que se tomara como verdadera (1)

        Returns:
            list: Lista con elementos 1 y 0, segun cumplan la condicion
        """
        for i in range(len(lista)):
            lista[i] = 1 if lista[i]==condicion else 0
        return lista

    def df_to_excel(self,dataframe,tienda,producto): #importa un dt a excel
        """Exporta un dataframe a un archivo .xlsx

        Args:
            dataframe (_dataframe_): Dataframe que se desea exportar
            tienda (_str_): Nombre de la tienda asociada al dataframe
            producto (_str_): Producto asociado al dataframe
        """
        dataframe.to_excel(f"scrapper_{tienda}_{producto}.xlsx")

    def imagen(self,path): #visualiza imagen (con url imagen)
        """"Ayuda a visualizar la URL como imagen.

        Args:
            path (_str_): URL de la imagen

        Returns:
            _str_: Representa elemento html
        """
        return f'<img src="{path}" width="250" >'
    

    def scrapping_walmart(self,producto:str) -> pd:
        """Web scrapper para el sitio de Walmart.

        Args:
            producto (str): producto del que se desea obtener dataframe

        Returns:
            pd: dataframe del producto
        """
        driver = webdriver.Chrome(service=Service("C://Users//basal//Downloads//chromedriver_win32//chromedriver.exe"))
        driver.implicitly_wait(15)
        #producto="the last of us"
        producto=producto.replace(" ","%20")
        driver.get(f"https://www.walmart.com.mx/productos?Ntt={producto}")
        driver.implicitly_wait(15)
        productos=driver.find_elements(By.CLASS_NAME,"product_container__3DSm4")

        lista_nombres=self.list_maker_class(len(productos),productos,"nav-link_navLink__2oJ29.product_name__1YFfY")
        lista_marcas=self.list_maker_class(len(productos),productos,"product_brand__1CsRr")
        lista_url_imagenes=self.list_maker_attribute(len(productos),productos,"img","src")
        lista_urls_productos=self.list_maker_attribute(len(productos),productos,"a","href")
        lista_precios=self.list_maker_class2(len(productos),productos,"product_price__2NBjj","text_text__3oq-D.text_large__K4EIS.text_bold__2Ptj-")

        data={"MARCA":lista_marcas,"PRODUCTO":lista_nombres,"PRECIO":lista_precios,"IMAGEN":lista_url_imagenes,"LINK":lista_urls_productos}
        df_walmart=pd.DataFrame(data)
        df_walmart.insert(0,"FECHA",datetime.now().strftime("%d/%m/%Y"))
        df_walmart.insert(1,"TIENDA","WALMART")
        df_walmart["PRECIO"]=df_walmart["PRECIO"].str.replace("$"," ")

        self.df_to_excel(df_walmart,"walmart",producto)

        display(HTML(df_walmart.to_html(escape=False,formatters=dict(IMAGEN=self.imagen))))



    def scrapping_claro(self,producto:str) -> pd:
        """Web scrapper para el sitio de Claro Shop.

        Args:
            producto (str): producto del que se desea obtener dataframe

        Returns:
            pd: dataframe del producto
        """
        driver = webdriver.Chrome(service=Service("C://Users//basal//Downloads//chromedriver_win32//chromedriver.exe"))
        driver.implicitly_wait(5)
        producto=producto.replace(" ","%20")
        driver.get(f"https://www.claroshop.com/resultados/q={producto}/pagina=1")
        driver.implicitly_wait(5)
        productos=driver.find_elements(By.CLASS_NAME,"cardProduct")

        lista_nombres=self.list_maker_class2(len(productos),productos,"contDataCard","h4")
        lista_urls_productos=self.list_maker_attribute(len(productos),productos,"a","href")
        lista_precios=self.list_maker_class2(len(productos),productos,"contDataCard","precio1")
        lista_precios_limpia=self.delete_extra_info(lista_precios," ")

        lista_envios=self.list_maker_class2(len(productos),productos,"expressFreeCont","envioGratis")
        lista_envios_binaria=self.to_binary(lista_envios,"Envío gratis")

        data={"PRODUCTO":lista_nombres,"PRECIO":lista_precios,"ENVIO":lista_envios_binaria,"LINK":lista_urls_productos}
        df_claro=pd.DataFrame(data)
        df_claro.insert(0,"FECHA",datetime.now().strftime("%d/%m/%Y"))
        df_claro.insert(1,"TIENDA","CLARO SHOP")
        df_claro["PRECIO"]=df_claro["PRECIO"].str.replace("$"," ")

        self.df_to_excel(df_claro,"claro_shop",producto)

        return df_claro


    def scrapping_game_planet(self,producto:str) -> pd:
        """Web scrapper para el sitio de Game Planet.

        Args:
            producto (str): producto del que se desea obtener dataframe

        Returns:
            pd: dataframe del producto
        """
        driver = webdriver.Chrome(service=Service("C://Users//basal//Downloads//chromedriver_win32//chromedriver.exe"))
        driver.implicitly_wait(5)
        producto=producto.replace(" ","%20")
        driver.get(f"https://gameplanet.com/?s={producto}&post_type=product&dgwt_wcas=1")
        driver.implicitly_wait(5)
        productos=driver.find_elements(By.CLASS_NAME,"product-small.box")

        lista_nombres=self.list_maker_class(len(productos),productos,"name.product-title.woocommerce-loop-product__title")
        lista_categorias=self.list_maker_class(len(productos),productos,"category.uppercase.is-smaller.no-text-overflow.product-cat.op-7")
        lista_estados=self.list_maker_class(len(productos),productos,"condicion.uppercase.is-smaller.no-text-overflow.product-condicion")
        lista_url_imagenes=self.list_maker_attribute(len(productos),productos,"img","src")
        lista_urls_productos=self.list_maker_attribute(len(productos),productos,"a","href")

        data={"CATEGORIAS":lista_categorias,"PRODUCTO":lista_nombres,"CONDICION":lista_estados,"IMAGEN":lista_url_imagenes,"LINK":lista_urls_productos}

        df_game_planet=pd.DataFrame(data)
        df_game_planet.insert(0,"FECHA",datetime.now().strftime("%d/%m/%Y"))
        df_game_planet.insert(1,"TIENDA","GAME PLANET")
        
        self.df_to_excel(df_game_planet,"game_planet",producto)

        df_game_planet
        display(HTML(df_game_planet.to_html(escape=False,formatters=dict(IMAGEN=self.imagen))))


In [2]:
WS_prueba4=Web_Scrapper()

WS_prueba4.scrapping_walmart("playstation 4")

C:\Users\basal\AppData\Local\Temp\ipykernel_8432\4284807482.py:155: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_walmart["PRECIO"]=df_walmart["PRECIO"].str.replace("$"," ")


,FECHA,TIENDA,MARCA,PRODUCTO,PRECIO,IMAGEN,LINK
0,16/12/2022,WALMART,playstation 5,Consola PlayStation 5 Estándar 825 GB,"12,599.00",,https://www.walmart.com.mx/videojuegos/playstation-5/consolas-ps5/consola-playstation-5-estandar-825-gb_00071171954889
1,16/12/2022,WALMART,playstation 5,Consola Digital PlayStation 5 Horizon Forbidd...,"11,599.00",,https://www.walmart.com.mx/videojuegos/playstation-5/consolas-ps5/consola-digital-playstation-5-horizon-forbidden-west_00071171955684
2,16/12/2022,WALMART,playstation 5,Consola Estándar PlayStation 5 God of War Rag...,"15,499.00",,https://www.walmart.com.mx/videojuegos/playstation-5/consolas-ps5/consola-estandar-playstation-5-god-of-war-ragnarok_00071171955534
3,16/12/2022,WALMART,playstation 5,Consola Estándar PlayStation 5 FIFA 23,"15,499.00",,https://www.walmart.com.mx/videojuegos/playstation-5/consolas-ps5/consola-estandar-playstation-5-fifa-23_00071171955894
4,16/12/2022,WALMART,playstation 5,God of War Ragnarök PlayStation 5 Físico,"1,699.00",,https://www.walmart.com.mx/videojuegos/playstation-5/juegos-ps5/god-of-war-ragnarok-playstation-5-fisico_00071171954817
5,16/12/2022,WALMART,playstation 5,FIFA 23 PlayStation 5 Físico,"1,249.00",,https://www.walmart.com.mx/videojuegos/playstation-5/juegos-ps5/fifa-23-playstation-5-fisico_00001463337929
6,16/12/2022,WALMART,playstation 4,FIFA 23 PlayStation 4 Físico,999.00,,https://www.walmart.com.mx/videojuegos/nintendo/juegos/fifa-23-playstation-4-fisico_00001463337927
7,16/12/2022,WALMART,playstation 4,God of War Ragnarök PlayStation 4 Físico,"1,399.00",,https://www.walmart.com.mx/videojuegos/playstation-4/juegos-ps4/god-of-war-ragnarok-playstation-4-fisico_00071171954815
8,16/12/2022,WALMART,playstation 5,Control Inalámbrico PlayStation 5 DualSense,"1,190.00",,https://www.walmart.com.mx/videojuegos/playstation-5/accesorios-ps5/control-inalambrico-playstation-5-dualsense_00071171954136
9,16/12/2022,WALMART,playstation 5,F1 22 PlayStation 5 Físico,999.00,,https://www.walmart.com.mx/videojuegos/playstation-5/juegos-ps5/f1-22-playstation-5-fisico_00001463374786


In [7]:
WS_prueba5=Web_Scrapper()

WS_prueba5.scrapping_walmart("plants vs zombies xbox one")

C:\Users\basal\AppData\Local\Temp\ipykernel_8432\4284807482.py:155: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_walmart["PRECIO"]=df_walmart["PRECIO"].str.replace("$"," ")


,FECHA,TIENDA,MARCA,PRODUCTO,PRECIO,IMAGEN,LINK
0,16/12/2022,WALMART,xbox one,Minecraft Story Mode Season 2 Xbox One Xbox O...,499.00,,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/minecraft-story-mode-season-2-xbox-one-xbox-one-standard_00088537082991
1,16/12/2022,WALMART,xbox one,Dead or Alive 6 Xbox One Xbox One Standard,579.00,,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/dead-or-alive-6-xbox-one-xbox-one-standard_00004019800306
2,16/12/2022,WALMART,xbox one,FORTNITE DEEP XBOX ONE Xbox One standard,"4,999.00",,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/fortnite-deep-xbox-one-xbox-one-standard_00088392966366
3,16/12/2022,WALMART,xbox one,MONSTER ENERGY SUPERCROSS 2 XBOX ONE Xbox One...,599.00,,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/monster-energy-supercross-2-xbox-one-xbox-one-standard_00066224892235
4,16/12/2022,WALMART,xbox one,RIDE 2 XBOX ONE Xbox One Standard,"1,149.00",,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/ride-2-xbox-one-xbox-one-standard_00066224891888
5,16/12/2022,WALMART,xbox one,Plants Vs. Zombies Garden Warfare 2 - Xbox On...,"1,499.00",,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/plants-vs-zombies-garden-warfare-2-xbox-one-xbox-one-game_00500016915261
6,16/12/2022,WALMART,xbox one,FIFA 21 XBOX ONE Xbox One Standard,"1,499.00",,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/fifa-21-xbox-one-xbox-one-standard_00001463337812
7,16/12/2022,WALMART,xbox one,RESIDENT EVIL 2 REVELATIONS XBOX ONE Xbox One...,849.00,,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/resident-evil-2-revelations-xbox-one-xbox-one-standard_00001338855011
8,16/12/2022,WALMART,xbox one,LATAMEL COD INFINITE WAR XBOX ONE Xbox One 2 ...,"1,499.00",,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/latamel-cod-infinite-war-xbox-one-xbox-one-2-en-1-call-of-duty-infinite-warfare-modern-warfare-1-pza_00004787588039
9,16/12/2022,WALMART,xbox one,Red Dead Redemption 2 Xbox One Xbox One Game,979.00,,https://www.walmart.com.mx/videojuegos/xbox-one/juegos/red-dead-redemption-2-xbox-one-xbox-one-game_00502655535897


In [8]:
WS_prueba6=Web_Scrapper()

WS_prueba6.scrapping_walmart("nintendo switch")

AttributeError: Can only use .str accessor with string values!